# Problem 6: Multilayer Perceptron

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
from utils.activations import softmax, sigmoid
from utils.data_parser import data_split_train_test
np.random.seed(1)

#define constants
num_class = 4 # no of classes
hidden_layer_units = 25  #number of units in the hidden layer
epochs = 200  #number of iterations
alpha = 0.001 #learning rate 


## DataSource
We split the provided data into 2 sets: training set and validation set with the ration 8:2

In [2]:
x = pd.read_csv(os.path.join('data','train_data.csv'), header=None).add_prefix('Feature_')
y = pd.read_csv(os.path.join('data', 'train_labels.csv'), header=None, names=["Label_0", "Label_1", "Label_2", "Label_3"])
X, Y, X_v, Y_v = data_split_train_test(x, y)
X_train = X.values   #training samples
Y_train = Y.values   #training labels

m, n = X_train.shape  #samples x features
X_Validate = X_v.values  #validation samples
Y_Validate = Y_v.values  #validation labels


## Layer, Forward Propagation, and Backward Propagation
We then implement Layer, Forward Propagation, and Back Propagation as follow
<img align="left" src="./system.png"     style=" width:380px; padding: 10px; " >


In [3]:
def Layer(A_in, W, B, g):
    """
    :param A_in: shape(m,n) - input data
    :param W: shape(feature,units) - weight matrix, n0 feature  x units, 
    :param b: shape(units,1) - bias vector, n0 units x 1
    :param g: activation function(e.g sigmoid, relu, softmax, ...)
    :return:
    Z - linear regression
    A_out: shape(m, units): output data - m x units
    """
    Z = np.dot(A_in, W) + B
    A_out = g(Z)
    return Z,A_out

In [4]:
def compute_forward_prop(x, W1, b1, W2, b2):
    z1, a1 = Layer(x, W1, b1, sigmoid)  #hidden layer [1] with sigmoid activation
    z2, a2 = Layer(a1, W2, b2, softmax) #output layer [2] with softmax activation
    return z1, a1, z2, a2

Back-propagation is computed by the formulas:
$$dz^{[2]}=a^{[2]} - Ytrain$$
$$dW^{[2]}=a^{[1]T}\cdot dz^{[2]}$$
$$db^{[2]}=dz^{[2]}$$
$$dz^{[1]}=dz^{[2]}  W^{[2]T}$$
$$dW^{[1]}=X^{T}\cdot dz^{[1]}*(z^{[1]})^{'}$$
$$db^{[1]}=dz^{[1]}*(z^{[1]})^{'}$$

In [5]:
def sigmoid_derivative(x): 
    return sigmoid(x) * (1 - sigmoid(x))

def compute_backward_prop(Z1, A1, A2, W2,X, Y): 
    dz2 = A2 - Y
    dW2 = np.dot(A1.T,dz2)
    db2 = dz2
    dz1 = np.dot(dz2,W2.T)
    db1 = dz1 * sigmoid_derivative(Z1)
    dW1 = np.dot(X.T,dz1 * sigmoid_derivative(Z1))
    return db1, dW1, db2, dW2

## Gradient descent 
Then, these values from back-prop will be fed into gradient descent algorithm to re-calculate the weights and bias, so that the system can minimize the cost function:
$$W^{[1]} = W^{[1]} - \alpha dW^{[1]}$$
$$b^{[1]} = b^{[1]} - \alpha db^{[1]}$$
$$W^{[2]} = W^{[2]} - \alpha dW^{[2]}$$
$$b^{[2]} = b^{[2]} - \alpha db^{[2]}$$

In [6]:
def update_model_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha_):
    W1 = W1 - alpha_ * dW1
    W2 = W2 - alpha_ * dW2
    b1 = b1 - alpha_ * db1.sum(axis=0)
    b2 = b2 - alpha_ * db2.sum(axis=0)
    return b1, W1, b2, W2

## Cost function
Cost function is defined by the average of square of error function
$$Cost = \frac{1}{m}\sum_{i=0}^{m}(\hat{y}-y)^2$$

In [7]:
def compute_cost(y_hat, y):
    return np.mean(np.square(y_hat - y))

## Initialize model parameters
The model parameters are intialized randomly as follow:

In [8]:

def initialize_model_params():
    W1 = np.random.randn(n, hidden_layer_units)
    b1 = np.random.randn(hidden_layer_units)
    W2 = np.random.randn(hidden_layer_units, num_class)
    b2 = np.random.randn(num_class)
    return b1, W1, b2, W2

## Accuracy and one-hot encode
In order to analyze the results, the implementation of prediction and accuracy are showed below:  

In [9]:
def predict(X, B1, W1, B2, W2):
    Z1 = np.dot(X, W1) + B1
    A1 = sigmoid(Z1)
    Z2 = np.dot(A1, W2) + B2
    A2 = softmax(Z2)
    prediction = np.argmax(A2, 0)
    return prediction

def compute_accuracy(Y_hat, Y):
    correct_count = sum((Y[i] == Y_hat[i]).all() for i in range(len(Y)))
    accuracy = correct_count / len(Y)
    return accuracy

def convert_to_one_hot(A):
    num_classes = A.shape[1]
    max_value = np.max(A) + 1
    one_hot_encoded = np.zeros((A.shape[0], num_classes), dtype=int)
    indices = np.argmax(A, axis=1)
    one_hot_encoded[np.arange(A.shape[0]), indices] = 1
    return one_hot_encoded

## Wrap everything up 
Sequence of training process: Compute forward prop -> compute backward prop -> compute gradient descend, then repeat until convergence  

In [10]:
def training_data(X, Y, X_v, Y_v, epochs, alpha):
    b1, W1, b2, W2 = initialize_model_params()
    accuracy = 0.
    for i in range(epochs):
        z1, a1, z2, a2 = compute_forward_prop(X, W1, b1, W2, b2)
        db1, dW1, db2, dW2 = compute_backward_prop(z1, a1, a2, W2, X, Y) 
        b1, W1, b2, W2 = update_model_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)

        #compute cost for each epoch        
        cost = compute_cost(a2, Y) 
        #avoid overfitting by using validation set for prediction
        _,_,_,y_hat = compute_forward_prop(X_v, W1, b1, W2, b2)
        #convert prediction to one-hot encode
        y_hat_encoded = convert_to_one_hot(y_hat)

        accuracy = compute_accuracy(y_hat_encoded, Y_v)
        if i % 10 == 0:
            print("Epoch: ", i)
            print(f"cost = {cost}  accuracy={accuracy * 100}")
    return b1, W1, b2, W2, accuracy * 100

Run the training process with $epochs=200, \alpha=0.001$:

In [11]:
b1, W1, b2, W2, accuracy = training_data(X_train, Y_train, X_Validate, Y_Validate, epochs, alpha)

Epoch:  0
cost = 0.3204854569629083  accuracy=40.537265198949704


/Users/quanquach/Desktop/ECE657/Assignment1/utils/activations.py:21: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))


Epoch:  10
cost = 0.37141426065680894  accuracy=23.20743284185013
Epoch:  20
cost = 0.2488774359580971  accuracy=44.29408200363562
Epoch:  30
cost = 0.18221521111925298  accuracy=70.63219551605737
Epoch:  40
cost = 0.12435261959095559  accuracy=70.6523934558675
Epoch:  50
cost = 0.13528077211669987  accuracy=71.1169460715007
Epoch:  60
cost = 0.030364072847876625  accuracy=91.55726115936174
Epoch:  70
cost = 0.03971068332376786  accuracy=92.68834578872955
Epoch:  80
cost = 0.02438133269940144  accuracy=94.64754595031307
Epoch:  90
cost = 0.025728821849999483  accuracy=93.49626338113512
Epoch:  100
cost = 0.02354410228287669  accuracy=93.75883659866695
Epoch:  110
cost = 0.022973567812959717  accuracy=93.83962835790749
Epoch:  120
cost = 0.022233967364524355  accuracy=94.04160775600889
Epoch:  130
cost = 0.019935447156864874  accuracy=95.55645324176933
Epoch:  140
cost = 0.019247927476592416  accuracy=94.52635831145223
Epoch:  150
cost = 0.019115202393230793  accuracy=95.83922439911129


The accuracy of the validation set is around 95%

In [12]:
accuracy

95.05150474651586

And the model parameters are:

In [13]:
b1, W1, b2, W2

(array([ -1.20879954,  -2.31890563,  -5.69133798,  -4.1128773 ,
         -0.50367965,  -2.05513728,  -1.14467325,  -2.42138646,
         -4.43622852,   0.63476919,  -4.8365037 ,  -3.62261245,
          0.70081257,  -5.42958767, -10.37903416,  -3.8499326 ,
         -2.93411032,  -3.57072241,  -6.13875778,  -2.88279477,
         -1.91591634,  -0.26224121,   0.11872386,  -0.05141358,
         -1.73938463]),
 array([[ 1.62434536, -0.61175641, -0.52817175, ...,  0.90159072,
          0.50249434,  0.90085595],
        [-0.68372786, -0.12289023, -0.93576943, ...,  2.10025514,
          0.12015895,  0.61720311],
        [ 0.30017032, -0.35224985, -1.1425182 , ...,  0.16003707,
          0.87616892,  0.31563495],
        ...,
        [-0.32425465,  1.06686631, -0.37814658, ...,  0.3938322 ,
         -0.15137905, -0.41505414],
        [-0.85782245,  0.47026103, -0.71104247, ..., -0.91428204,
          0.80115669,  0.11465216],
        [-2.30445222,  1.69949757, -0.76992763, ..., -0.51091193,
   

Finally, we can save them for later prediction

In [14]:
np.save('hidden_weight.npy', W1)
np.save('hidden_bias.npy', b1)
np.save('output_weight.npy', W2)
np.save('output_bias.npy', b2)

# Conclusion

In [15]:
# X_test = np.random.rand(1, n)
# print(X_test.shape)
# prediction = predict(X_test, b1, W1, b2, W2)
# print(f"prediction = {prediction}")
# print(f"X_test = {X_test}")